In [1]:
import numpy as np
import random

In [11]:
class Simulation:
    def __init__(self, evaluation_function):
        self.calculate_fitness = evaluation_function
        self.best_agents = [([], float('-inf'), -1)] #Agent, fitness, epoch [([], float('-inf'), -1)]
        
    def calculate_fitness_array(self, agents):
        fitnesses = np.array([self.calculate_fitness(agent) for agent in agents])
        return fitnesses
    
    def calculate_diversity_score_array(self, agents):
        uniqueness = []
        for agent in agents:
            dists = np.sum([agent != x for x in agents])
            #dists = np.array([np.linalg.norm(agent-x) for x in agents])
            if np.any(dists):
                uniqueness.append(min(dists[dists != 0]))
            else:
                uniqueness.append(0)
            
        return np.array(uniqueness)
        #Old Code: Probably cheaper though
        #return np.array([self.distance(np.mean(agents, axis=0), agent) for agent in agents])

    def calculate_reproduction_chance(self, population, value_uniqueness = 0.5):
        fitnesses = self.calculate_fitness_array(population.agents)
        non_norm_fitnesses = np.copy(fitnesses)
        
        #print(self.best_agents)
        if max(fitnesses) > self.best_agents[-1][1]:
            self.best_agents.append((population.agents[np.argmax(fitnesses)], max(fitnesses), population.epoch))
        
        fitnesses = (fitnesses-np.mean(fitnesses))/np.std(fitnesses)

        if value_uniqueness > 0:
            gene_uniqueness = self.calculate_diversity_score_array(population.agents)#np.array([self.distance(np.mean(population.agents, axis=0), agent) for agent in population.agents])
            gene_uniqueness = (gene_uniqueness-np.mean(gene_uniqueness))/np.std(gene_uniqueness)

            reproduction_chance = fitnesses + (gene_uniqueness * value_uniqueness)
            reproduction_chance = (reproduction_chance-np.mean(reproduction_chance))/np.std(reproduction_chance)

            return reproduction_chance+2, non_norm_fitnesses

        return fitnesses+2, non_norm_fitnesses

    def distance(self, x1, x2):
        return np.linalg.norm(x1-x2)

    def mutate_agent(self, agent, rate, bounds):

        if rate < 1/1000000:
            return agent

        mutations = np.random.uniform(0,1/rate, size=agent.shape)
        #mutations[mutations>1] = 0
        
        #print(agent[mutations <= 1].shape, (mutations <= 1).shape)
        
        np.putmask(agent, mutations <= 1, np.round(np.random.random_sample(size=(mutations <= 1).shape)).astype(int)) #np.random.randint(0, 2, size = len(mutations <= 1))

        return agent.astype(int)

    def mutate_array(self, agents, rate, bounds):
        mutated_agents = np.array([self.mutate_agent(agent, rate, bounds) for agent in agents])
        return mutated_agents

    def merge_genes(self, a, b, n_genes):
        merged = np.zeros(shape=(n_genes, a.shape[0], int(a.shape[1]/n_genes)))
    
        genes_a = a.reshape(n_genes, a.shape[0], int(a.shape[1]/n_genes))
        genes_b = b.reshape(n_genes, a.shape[0], int(b.shape[1]/n_genes))
        to_merge = np.random.randint(0,2, size=(n_genes))

        merged[to_merge == 0] = genes_a[to_merge == 0]
        merged[to_merge == 1] = genes_b[to_merge == 1]

        return merged.reshape(a.shape[0], a.shape[1])

    def next_generation(self, population, mutation_rate=0.01, value_diversity = 0.5, show_fitness = False, show_diversity = False):

        fitnesses, fitness_array = self.calculate_reproduction_chance(population, value_uniqueness=value_diversity)
        gene_pool = np.array(random.choices(population.agents, weights = fitnesses, k=population.agents.shape[0]*2))

        next_gen = np.array([self.merge_genes(gene_pool[i], gene_pool[i+1], population.num_genes) for i in range(int(len(gene_pool)/2))])
        next_gen = self.mutate_array(next_gen, mutation_rate, population.bounds)
        
        if show_fitness and show_diversity:
            fitness = np.mean(fitness_array)
            diversity = np.mean(self.calculate_diversity_score_array(population.agents))
            return next_gen, fitness, diversity
        if show_fitness:
            fitness = np.mean(fitness_array)
            return next_gen, fitness
        if show_diversity:
            diversity = np.mean(self.calculate_diversity_score_array(population.agents))
            return next_gen, diversity
        return next_gen